In [1]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.6 MB/s eta 0:00:00


In [2]:
!pip install statsmodels

In [3]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from prophet import Prophet
from ipywidgets import interact, interactive
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def cargar_datos(ruta, columna_fecha='fecha'):
    df = pd.read_csv(ruta)
    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    return df[df[columna_fecha] >= '2000-01-01']

In [6]:
ruta_base = '/content/drive/MyDrive/AguaMx/'
almacenamiento = cargar_datos(ruta_base + 'almacenamiento.csv')
area = cargar_datos(ruta_base + 'area.csv')
elevacion = cargar_datos(ruta_base + 'elevacion.csv')
evaporacion = cargar_datos(ruta_base + 'evaporacion.csv')
extracciones_toma = cargar_datos(ruta_base + 'extracciones_por_obra_de_toma.csv')
extracciones_vertedor = cargar_datos(ruta_base + 'extracciones_por_vertedor.csv')
precipitacion = cargar_datos(ruta_base + 'precipitacion.csv')


In [7]:
datasets = {
    'Precipitación': precipitacion,
    'Evaporación': evaporacion,
    'Extracciones por Obra de Toma': extracciones_toma,
    'Extracciones por Vertedor': extracciones_vertedor
}

In [8]:
def hacer_prediccion_arima(datos, columna_presa, pasos_prediccion=12):
    presa_data = datos[['fecha', columna_presa]].set_index('fecha').dropna()

    presa_data.plot(title=f'Datos históricos de {columna_presa}')
    plt.show()
    model = ARIMA(presa_data, order=(5,1,0))
    model_fit = model.fit()

    predicciones = model_fit.forecast(steps=pasos_prediccion)

    plt.plot(predicciones, label='Predicciones')
    plt.title(f'Predicciones de {columna_presa} para los próximos {pasos_prediccion} meses')
    plt.legend()
    plt.show()

    return predicciones

In [9]:
dataset_selector = widgets.Dropdown(
    options=list(datasets.keys()),
    description='Dataset:',
    disabled=False,
)

In [10]:

presa_selector = widgets.Dropdown(
    options=precipitacion.columns[1:],
    description='Presa:',
    disabled=False,
)

In [11]:
boton_prediccion = widgets.Button(description="Predecir")

In [12]:
def on_button_click(b):

    dataset_name = dataset_selector.value
    presa_name = presa_selector.value


    dataset_seleccionado = datasets[dataset_name]


    hacer_prediccion_arima(dataset_seleccionado, presa_name)

boton_prediccion.on_click(on_button_click)

display(dataset_selector, presa_selector, boton_prediccion)


Dropdown(description='Dataset:', options=('Precipitación', 'Evaporación', 'Extracciones por Obra de Toma', 'Ex…

Dropdown(description='Presa:', options=('ABRAG', 'ACHZC', 'AFIGR', 'AGCSO', 'AGMNR', 'AGOMC', 'AGZCH', 'ALMSI'…

Button(description='Predecir', style=ButtonStyle())

Con Prophet

In [13]:
ruta_base = '/content/drive/MyDrive/AguaMx/'

def cargar_datos(ruta, columna_fecha='fecha'):
    df = pd.read_csv(ruta)
    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    df.set_index(columna_fecha, inplace=True)
    return df

precipitacion = cargar_datos(ruta_base + 'precipitacion.csv')
evaporacion = cargar_datos(ruta_base + 'evaporacion.csv')
extracciones_toma = cargar_datos(ruta_base + 'extracciones_por_obra_de_toma.csv')
extracciones_vertedor = cargar_datos(ruta_base + 'extracciones_por_vertedor.csv')

datasets = {
    'Precipitación': precipitacion,
    'Evaporación': evaporacion,
    'Extracciones por Obra de Toma': extracciones_toma,
    'Extracciones por Vertedor': extracciones_vertedor
}

def obtener_presas(dataset_name):
    datos = datasets[dataset_name]
    return list(datos.columns)


def aplicar_prophet_interactivo(dataset_name, presa_name, dias_prediccion=365, anio_inicio=2000):
    datos = datasets[dataset_name]

    datos_filtrados_por_anio = datos[datos.index.year >= anio_inicio]

    if datos_filtrados_por_anio.empty:
        print(f"No hay suficientes datos en {dataset_name} después del año {anio_inicio}.")
        return

    if presa_name not in datos_filtrados_por_anio.columns:
        print(f"La presa {presa_name} no está disponible en {dataset_name}.")
        return

    df_prophet = datos_filtrados_por_anio.reset_index().rename(columns={'fecha': 'ds', presa_name: 'y'}).dropna(subset=['y'])

    if df_prophet.empty:
        print(f"No hay datos válidos para la presa {presa_name}.")
        return

    modelo = Prophet()
    modelo.fit(df_prophet)

    futuro = modelo.make_future_dataframe(periods=dias_prediccion)
    predicciones = modelo.predict(futuro)

    fig = modelo.plot(predicciones)
    plt.title(f"Predicciones para {presa_name} en {dataset_name}")
    plt.show()

    fig2 = modelo.plot_components(predicciones)
    plt.show()

def seleccionar_presa(dataset_name):
    presas = obtener_presas(dataset_name)
    presa_widget = widgets.Dropdown(
        options=presas,
        description='Presa',
        value=presas[0]
    )
    return presa_widget

def interfaz_prophet():
    dataset_widget = widgets.Dropdown(
        options=list(datasets.keys()),
        value='Precipitación',
        description='Dataset'
    )

    def actualizar_presa_widget(dataset_name):
        return seleccionar_presa(dataset_name)

    presa_widget = seleccionar_presa('Precipitación')

    dataset_presa = interactive(
        lambda dataset_name: seleccionar_presa(dataset_name),
        dataset_name=dataset_widget
    )

    interact(
        aplicar_prophet_interactivo,
        dataset_name=dataset_widget,
        presa_name=presa_widget,
        dias_prediccion=widgets.IntSlider(min=30, max=730, step=30, value=365, description="Días de Predicción"),
        anio_inicio=widgets.IntSlider(min=1990, max=2023, step=1, value=2000, description="Año de Inicio")
    )

interfaz_prophet()


interactive(children=(Dropdown(description='Dataset', options=('Precipitación', 'Evaporación', 'Extracciones p…

Clasificación de series de tiempo

In [14]:
!pip install tslearn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 6.5 MB/s eta 0:00:00


In [15]:
ruta_base = '/content/drive/MyDrive/AguaMx/'

def cargar_datos(ruta, columna_fecha='fecha'):
    df = pd.read_csv(ruta)
    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    df.set_index(columna_fecha, inplace=True)
    return df

precipitacion = cargar_datos(ruta_base + 'precipitacion.csv')
evaporacion = cargar_datos(ruta_base + 'evaporacion.csv')
extracciones_toma = cargar_datos(ruta_base + 'extracciones_por_obra_de_toma.csv')
extracciones_vertedor = cargar_datos(ruta_base + 'extracciones_por_vertedor.csv')

datasets = {
    'Precipitación': precipitacion,
    'Evaporación': evaporacion,
    'Extracciones por Obra de Toma': extracciones_toma,
    'Extracciones por Vertedor': extracciones_vertedor
}

def aplicar_kmeans_interactivo(dataset_name, n_muestras=50, n_clusters=4, min_datos_validos=100, anio_inicio=2000):
    datos = datasets[dataset_name]


    datos_filtrados_por_anio = datos[datos.index.year >= anio_inicio]

    columnas_validas = datos_filtrados_por_anio.columns[datos_filtrados_por_anio.notna().sum() > min_datos_validos]

    if len(columnas_validas) == 0:
        print("No hay columnas con suficientes datos válidos.")
        return

    datos_filtrados = datos_filtrados_por_anio[columnas_validas]

    datos_filtrados = datos_filtrados.fillna(datos_filtrados.mean())

    muestra_presas = datos_filtrados.sample(n=n_muestras, axis=1, random_state=42)

    print(f"Columnas válidas seleccionadas: {len(muestra_presas.columns)}")

    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(muestra_presas.values.T)

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(data_scaled)

    presas = muestra_presas.columns
    clusters = pd.DataFrame({'Presa': presas, 'Cluster': labels})

    display(clusters)

    for cluster_num in range(n_clusters):
        plt.figure(figsize=(10, 6))
        for i, label in enumerate(labels):
            if label == cluster_num:
                plt.plot(muestra_presas.index, muestra_presas.iloc[:, i], label=presas[i])
        plt.title(f'Cluster {cluster_num + 1}')
        plt.legend(loc='best')
        plt.show()

interact(
    aplicar_kmeans_interactivo,
    dataset_name=widgets.Dropdown(
        options=list(datasets.keys()),
        value='Precipitación',
        description='Dataset'
    ),
    n_muestras=widgets.IntSlider(min=10, max=200, step=10, value=50, description="Muestras"),
    n_clusters=widgets.IntSlider(min=2, max=10, step=1, value=4, description="Clusters"),
    min_datos_validos=widgets.IntSlider(min=10, max=1000, step=10, value=100, description="Min Datos Válidos"),
    anio_inicio=widgets.IntSlider(min=1990, max=2023, step=1, value=2000, description="Año de Inicio")
)


interactive(children=(Dropdown(description='Dataset', options=('Precipitación', 'Evaporación', 'Extracciones p…

<function __main__.aplicar_kmeans_interactivo(dataset_name, n_muestras=50, n_clusters=4, min_datos_validos=100, anio_inicio=2000)>